In [1]:
import pandas as pd

In [5]:
dataset=pd.read_json('data/All_Beauty_5.json', lines=True)
dataset.head(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN
5,5,False,"05 16, 2010",A24HQ2N7332W7W,B00006L9LC,{'Size:': ' 366'},Kindle Customer Joyce Wilson,"If you know the scent of Diva, you'll LOVE thi...",Diva is Heavenly,1273968000,NaN,NaN
6,5,False,"05 7, 2018",A2G90R2ZU6KU5D,B00006L9LC,{'Size:': ' Small'},Mike,Got this shampoo as a solution for my wife's d...,"Outstanding, no complains",1525651200,NaN,NaN
7,2,True,"05 7, 2018",A24W4W9E62FZP2,B00006L9LC,{'Size:': ' Small'},Reb,No change my scalp still itches like crazy. It...,No change my scalp still itches like crazy. It...,1525651200,NaN,NaN
8,1,True,"05 6, 2018",A7ID5H7FWLJHC,B00006L9LC,{'Size:': ' Small'},U. V.,Too expensive for such poor quality. There was...,Too expensive for such poor quality. There was...,1525564800,NaN,NaN
9,1,True,"05 6, 2018",AYKOSAJTP5AVS,B00006L9LC,{'Size:': ' Small'},Senthil Kumar M,"It dries my hair, doesnt help to reduce dandru...","Dries my hair, doesnt help to reduce dandruff....",1525564800,NaN,NaN


In [7]:
dataset['style']

0       {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...
1       {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...
2       {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...
3       {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...
4                               {'Size:': ' 200ml/6.7oz'}
                              ...                        
5264                       {'Design:': ' ETA-C6B5F7C374'}
5265                       {'Design:': ' ETA-7117EE3788'}
5266                       {'Design:': ' ETA-BF66BD2F87'}
5267                       {'Design:': ' ETA-03868FCB34'}
5268                       {'Design:': ' ETA-C4893097E9'}
Name: style, Length: 5269, dtype: object

In [18]:
dataset.drop(['style','image'], axis=1).describe(include='all') #style and image are causing issues with describe()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote
count,5269.000000,5269,5269,5269,5269,5269,5264,5264,5.269000e+03,552
unique,NaN,2,883,991,85,905,1363,1011,NaN,24
top,NaN,True,"07 14, 2016",AKJHHD5VEH7VG,B0012Y0ZG2,Amazon Customer,Great product,Five Stars,NaN,2
freq,NaN,4280,33,12,1794,362,15,1455,NaN,238
mean,4.771873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.441506e+09,NaN
std,0.743204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.473127e+07,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.095725e+09,NaN
25%,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.407024e+09,NaN
50%,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.445645e+09,NaN
75%,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.481674e+09,NaN


The dataset contains 5264 reviews for 85 Beauty products from 991 reviewers